In [4]:
# Import packages and set numpy random seed
import numpy as np
import cv2
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.models import Sequential
import matplotlib.cm as cm
import tensorflow as tf



In [5]:


model = Sequential(
[
    keras.Input((128,128,3)),
    Conv2D(32, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(32, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.5),

    Conv2D(16, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(8, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.5),

    Conv2D(4, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(1, activation='sigmoid')
]
)

optimizer = keras.optimizers.Adam(learning_rate=0.01)
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True) #from_logits=True means output probabilities are not normalized
acc_metric = keras.metrics.CategoricalAccuracy()
val_acc_metric = keras.metrics.BinaryAccuracy()

model.summary()
model.load_weights('data_aug_off_model.h5')


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 128, 128, 32)      2432      
_________________________________________________________________
batch_normalization_5 (Batch (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 64, 32)        25632     
_________________________________________________________________
batch_normalization_6 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 32, 32)       

In [6]:
@tf.function
def test_step(x):
    val_preds = model(x, training=False)    
    return val_preds

In [7]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [8]:
def display_gradcam(img, heatmap, alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img('img.jpg')
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    
    
    superimposed_img.save('img.jpg')
    return superimposed_img

    # Display Grad CAM

In [10]:
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray=[gray,gray,gray]
    to_test = np.resize(gray,(1,128,128,3))
#     print(to_test.shape)
    result = test_step(to_test)
    result = np.reshape(tf.get_static_value(result),(1,1))[0][0]
    
    # Display the resulting frame
    prediction = ''
    if result>0.5:
        prediction = 'non-drowsy'
    else:
        prediction = 'drowsy'
    
    cv2.imwrite('img.jpg', frame)
    heatmap = make_gradcam_heatmap(tf.expand_dims(np.resize(gray,(128,128,3)),0), model, 'conv2d_9')
    frame = display_gradcam(np.resize(frame,(128,128,3)), heatmap)
    frame = np.array(frame)[:, :, ::-1].copy()
    cv2.putText(frame, str(result), (10, 160), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 3) 
    cv2.putText(frame, prediction, (10, 200), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 3) 
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()